In [ ]:
# Upgrade pip
!pip install --upgrade pip

# Install or upgrade individual packages
!pip install --upgrade langchain openai chromadb pypdf tiktoken panel notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.4/502.4 kB 37.3 MB/s eta 0:00:00
   ━

In [ ]:
!pip install langchain openai chromadb tiktoken pypdf panel

In [ ]:
import os
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import panel as pn
import tempfile


In [ ]:
pn.extension('texteditor', template="bootstrap", sizing_mode='stretch_width')
pn.state.template.param.update(
    main_max_width="690px",
    header_background="#F08080",
)

In [ ]:
file_input = pn.widgets.FileInput(width=300)

openaikey = pn.widgets.PasswordInput(
    value="", placeholder="Your OpenAI API Key goes here.", width=300
)
prompt = pn.widgets.TextEditor(
    value="", placeholder="Ask me about the content of the PDF!", height=160, toolbar=False
)
run_button = pn.widgets.Button(name="Run!")

select_k = pn.widgets.IntSlider(
    name="Number of relevant chunks", start=1, end=5, step=1, value=2
)
select_chain_type = pn.widgets.RadioButtonGroup(
    name='Chain type',
    options=['stuff', 'map_reduce', "refine", "map_rerank"],
    value='map_reduce'
)

widgets = pn.Row(
    pn.Column(prompt, run_button, margin=5),
    pn.Card(
        "Chain type:",
        pn.Column(select_chain_type, select_k),
        title="Advanced settings"
    ), width=670
)

In [ ]:
def qa(file, query, chain_type, k):
    loader = PyPDFLoader(file)
    documents = loader.load()
    # split the documents into chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    embeddings = OpenAIEmbeddings()
    db = Chroma.from_documents(texts, embeddings)
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    qa = RetrievalQA.from_chain_type(
        llm=OpenAI(), chain_type=chain_type, retriever=retriever, return_source_documents=True)
    result = qa({"query": query})
    print(result['result'])
    return result

In [ ]:
import os

convos = []  # store all panel objects in a list

def qa_result(_):
    os.environ["OPENAI_API_KEY"] = openaikey.value

    # Check if the directory exists, and create it if not
    cache_dir = "/.cache/"
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)

    # Save pdf file to a temp file
    if file_input.value is not None:
        file_path = os.path.join(cache_dir, "temp.pdf")
        file_input.save(file_path)

        prompt_text = prompt.value
        if prompt_text:
            result = qa(file=file_path, query=prompt_text, chain_type=select_chain_type.value, k=select_k.value)
            convos.extend([
                pn.Row(
                    pn.panel("\U0001F60A", width=10),
                    prompt_text,
                    width=600
                ),
                pn.Row(
                    pn.panel("\U0001F916", width=10),
                    pn.Column(
                        result["result"],
                        "Relevant source text:",
                        pn.pane.Markdown('\n--------------------------------------------------------------------\n'.join(doc.page_content for doc in result["source_documents"]))
                    )
                )
            ])
    return pn.Column(*convos, margin=15, width=575, min_height=400)

In [ ]:
qa_interactive = pn.panel(
    pn.bind(qa_result, run_button),
    loading_indicator=True,
)

In [ ]:
output = pn.WidgetBox("*I'll answer your by writing here...:*", qa_interactive, width=670, scroll=True)

In [ ]:
# @title
# layout
pn.Column(
    pn.pane.Markdown("""
    ## \U0001F916¡ YucaPDF \U0001F916!
    ### \U0001F680 By: MauDev \U0001F680
    # Rules to use YucaPDF

    1) Upload a PDF.
    2) Enter OpenAI API key.
    3) Type a question and then just hit click the "Run"Button

    """),
    pn.Row(file_input,openaikey),
    output,
    widgets

).servable()

Column(sizing_mode='stretch_width')
    [0] Markdown(str, sizing_mode='stretch_width')
    [1] Row(sizing_mode='stretch_width')
        [0] FileInput(width=300)
        [1] PasswordInput(placeholder='Your OpenAI A...)
    [2] WidgetBox(css_classes=['panel-widget-box', ...], scroll=True, width=670)
        [0] Markdown(str, sizing_mode='stretch_width')
        [1] ParamFunction(function, _pane=Column, defer_load=False, loading_indicator=True, sizing_mode='stretch_width')
    [3] Row(width=670)
        [0] Column(margin=5, sizing_mode='stretch_width')
            [0] TextEditor(height=160, placeholder='Ask me about t..., sizing_mode='stretch_width', toolbar=False)
            [1] Button(name='Run!', sizing_mode='stretch_width')
        [1] Card(sizing_mode='stretch_width', title='Advanced settings')
            [0] Markdown(str, sizing_mode='stretch_width')
            [1] Column(sizing_mode='stretch_width')
                [0] RadioButtonGroup(name='Chain type', options=['stuff', 'map_reduce', ...], sizing_mode='stretch_width', value='map_reduce')
                [1] IntSlider(end=5, name='Number of relevant c..., sizing_mode='stretch_width', start=1, value=2)